In [1]:
from hyperopt import fmin, tpe, hp

In [ ]:
x = 4
y = 4
input_len = len(data[1])
iterations = 10000
sigma = 1
learning_rate = 0.001

In [ ]:
def train_som(x, y, input_len, sigma, learning_rate):
    
    # initialization of x*y SOM
    som = MiniSom(x=x, 
                  y=y, 
                  input_len=input_len, 
                  sigma=sigma, 
                  learning_rate=learning_rate)
    som.random_weights_init(data)
    
    # training
    start_time = time.time()
    som.train_random(data,iterations)
    elapsed_time = time.time() - start_time
    #print(elapsed_time, "seconds")
    #print('total_q_error :', som.quantization_error(data))
    
    return som

In [ ]:
som = train_som(x, y, input_len, sigma, learning_rate)

In [ ]:
for i in range(1, 10, 1) :
    for j in range(1, 10, 1):
        som = train_som(x, y, input_len, i/10, j/10)
        print('s = {}, lt = {}, q = {}'.format(i/10, j/10, np.round(som.quantization_error(data),3)))

In [ ]:
def plot_som(som):
    plt.figure(figsize=(6, 6))
    plt.imshow(som.distance_map()) #normalised sum of the distances between a neuron and its neighbours (euclidean distance)
    plt.colorbar()
    ax = plt.gca()
    ax.set_xticks(np.arange(-.5, 4, 1))
    ax.set_yticks(np.arange(-.5, 4, 1))
    ax.set_xticklabels(np.arange(0, 5, 1))
    ax.set_yticklabels(np.arange(0, 5, 1))

In [ ]:
som.distance_map()

In [ ]:
plot_som(som)

In [ ]:
start_time = time.time()
best = fmin(
    fn=lambda sig: MiniSom(x=x, 
              y=y, 
              input_len=input_len, 
              sigma=sig, 
              learning_rate=learning_rate).quantization_error(data),
    space=hp.uniform("sig", 0.001, x / 2.01),
    algo=tpe.suggest,
    max_evals=200)
elapsed_time = time.time() - start_time
print(elapsed_time, "seconds")
print(best)

In [ ]:
sigma = best['sig']
print("x: {}\ny: {}\ninput_len: {}\nsigma: {}\nlearning_rate: {}".format(x,y,len(data[1]),sigma,learning_rate))

In [ ]:
som = train_som(x, y, input_len, sigma, learning_rate)
plot_som(som)

In [ ]:
from hyperopt import Trials, STATUS_OK

space={
        'sig': hp.uniform('sig', 0.001, 5),
        'learning_rate': hp.uniform('learning_rate', 0.001, 5)
    }
def som_fn(space):
    sig = space['sig']
    learning_rate = space['learning_rate']
    val = MiniSom(x=x, 
                  y=y, 
                  input_len=input_len, 
                  sigma=sig, 
                  learning_rate=learning_rate).quantization_error(data)
    #print(val)
    return {'loss': val, 'status': STATUS_OK}

trials =  Trials()
best = fmin(fn=som_fn,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print('best: {}'.format(best))
for i, trial in enumerate(trials.trials[:2]):
    print(i, trial)

In [ ]:
sigma = best['sig']
learning_rate = best['learning_rate']
print("x: {}\ny: {}\ninput_len: {}\nsigma: {}\nlearning_rate: {}".format(x,y,len(data[1]),sigma,learning_rate))

In [ ]:
som = train_som(x, y, input_len, sigma, learning_rate)
plot_som(som)

In [ ]:
### Loop to test sigma and learning rate :
for i in range(1, 10, 1) :
    for j in range(1, 100, 1):
        som = train_som(x, y, input_len, i/10, j/100)
        print('s = {}, lt = {}, q = {}'.format(i/10, j/100, np.round(som.quantization_error(data),3)))